# Basic API workflow for UMLS

This notebook aims to get down the core functions which will make interacting with UMLS much more easy.

The goal is to:

- Authenticate
- Generate single use tickets
- Be able to do searches for specific terms
- Be able to view the semantic`semanticTypeGroup`

In [29]:
import requests
import json

In [42]:
MY_API_KEY = "c1d74116-7c2b-49b2-b2f4-e67edc1a1b21"
BASE_URL = "https://uts-ws.nlm.nih.gov"
TGT = "TGT-2053-cM07n0EvzCU0VVaYNtwfIVuQZgR4sPZJudErdlFbDzNQuboEcu-cas" # CHANGE THIS EVERY 8 HOURS

### Authentication

We need to authenticate using our API key, which will allow us to generate tickets. Each ticket needs to be generated every time a new API request is made (the tickets are single use unfortunately). For now the TGT value being passed in is hardcoded via the `TGT` variable. This is because the POST request route for TGT does not return JSON, but rather it returns HTML. Thought it would be possible to parse out the TGT, it is much easier right now to simply generate a new TGT manually, and then bring it into the Python code.

In [90]:
def get_ticket():
    """
    Get back a single use ticket. Needs to be used for every API request
    Returns: String for the new ticket
    """
    AUTH_URL = "https://utslogin.nlm.nih.gov"

    url = "{}/cas/v1/api-key/{}".format(AUTH_URL,TGT)
    try:
        res = requests.post(url, data={"service": "http://umlsks.nlm.nih.gov"})
    except requests.exceptions.HTTPError as e:
        print("Error getting a new ticket. Most likely the TGT needs to be refreshed.")
        print(e)
        return
    return res.text

In [91]:
get_ticket()

'ST-10680-wkb0mcnTZoVRrqntyaLb-cas'

### Searching

Being able to search will need to be one of the core functions of this API. This `search()` will allow us to do just that.

In [97]:
def search(term):
    """
    Able to search for a specific term in the UMLS database
    Param: term - search term
    Returns: A python object converted from the returned JSON. The search results
    """
    if not term:
        raise ValueError('Please provide a search term as a param to the function')
        return
    
    ticket = get_ticket() # Need to get a fresh ticket every time
    
    url = "{}/rest/search/current".format(BASE_URL)
    res = requests.get(url, params={"string": term, "searchType": "words", "ticket": ticket})
    return res.json()

In [98]:
suicide_search = search("suicide")

In [99]:
suicide_search

{'pageSize': 25,
 'pageNumber': 1,
 'result': {'classType': 'searchResults',
  'results': [{'ui': 'C0038661',
    'rootSource': 'MTH',
    'uri': 'https://uts-ws.nlm.nih.gov/rest/content/2018AA/CUI/C0038661',
    'name': 'Suicide'},
   {'ui': 'C0812393',
    'rootSource': 'MTH',
    'uri': 'https://uts-ws.nlm.nih.gov/rest/content/2018AA/CUI/C0812393',
    'name': 'Cancer patients and suicide and depression'},
   {'ui': 'C0852733',
    'rootSource': 'MTH',
    'uri': 'https://uts-ws.nlm.nih.gov/rest/content/2018AA/CUI/C0852733',
    'name': 'Completed Suicide'},
   {'ui': 'C0418299',
    'rootSource': 'SNOMEDCT_US',
    'uri': 'https://uts-ws.nlm.nih.gov/rest/content/2018AA/CUI/C0418299',
    'name': 'Suicide or attempted suicide by hanging'},
   {'ui': 'C0418301',
    'rootSource': 'SNOMEDCT_US',
    'uri': 'https://uts-ws.nlm.nih.gov/rest/content/2018AA/CUI/C0418301',
    'name': 'Suicide or attempted suicide by drowning'},
   {'ui': 'C0851163',
    'rootSource': 'MTH',
    'uri': 'ht

### Semantic Groupings

The process of semantic groupings involves looking up information on the term, and getting a TUI code. The TUI code is the identifier which can then be referenced in the semantic network.

So what we need to do is:

- Look up a term using its `ui` code (called the CUI) and get the TUI code from the URL of semanticTypes
- Get the semantic types from the semantics api end point

In [78]:
def get_tui(cui):
    """
    Given a CUI code, this function gets the TUI for the semantic groupings
    Params: ui - the CUI code for a specific term
    Returns: tui - string of the TUI id
    """
    if not cui:
        raise ValueError('Please provide a CUI code for a specific term')
        return
    
    ticket = get_ticket() # Need to get a fresh ticket every time
    
    url = "{}/rest/content/current/CUI/{}".format(BASE_URL, cui)
    res = requests.get(url, params={"ticket":ticket})
    obj = res.json()
    return obj['result']['semanticTypes'][0]['uri'].split('TUI/')[1]

In [101]:
get_tui("C0812393")

'T048'

In [92]:
def get_semantic_info(tui):
    """
    Provided a TUI id, this function gets the semantic type information for a specific TUI
    Params: TUI - string for the TUI
    Returns: object - the python object for the response for the API request
    """
    if not tui:
        raise ValueError('Please provide a TUI code')
        return
    
    ticket = get_ticket() # Need to get a fresh ticket every time
    
    url = "{}/rest/semantic-network/current/TUI/{}".format(BASE_URL, tui)
    res = requests.get(url, params={"ticket":ticket})
    return res.json()

In [102]:
semantic_info = get_semantic_info("T048")
semantic_info

{'pageSize': 25,
 'pageNumber': 1,
 'pageCount': 1,
 'result': {'classType': 'SemanticType',
  'abbreviation': 'mobd',
  'ui': 'T048',
  'definition': 'A clinically significant dysfunction whose major manifestation is behavioral or psychological. These dysfunctions may have identified or presumed biological etiologies or manifestations.',
  'example': 'NONE',
  'nonHuman': 'NONE',
  'usageNote': 'NONE',
  'treeNumber': 'B2.2.1.2.1.1',
  'semanticTypeGroup': {'classType': 'SemanticGroup',
   'abbreviation': 'DISO',
   'expandedForm': 'Disorders',
   'semanticTypeCount': 12},
  'childCount': 0,
  'name': 'Mental or Behavioral Dysfunction'}}

In [103]:
semantic_info['result']['semanticTypeGroup']

{'classType': 'SemanticGroup',
 'abbreviation': 'DISO',
 'expandedForm': 'Disorders',
 'semanticTypeCount': 12}

Look into relations: https://documentation.uts.nlm.nih.gov/rest/relations/
For the various relationships: https://www.nlm.nih.gov/research/umls/knowledge_sources/metathesaurus/release/abbreviations.html
Gathering the synonyms